In [ ]:
#@title Give a name to your project
TARGET = "object-detection" #@param {type:"string"}

In [ ]:
#@title Where to get your annotated dataset from?
DATASET_DOWNLOAD_LINK = "" #@param {type:"string"}

In [ ]:
#@title Where to get your pretrained model from?
MODEL_DOWNLOAD_LINK = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz" #@param {type:"string"}


In [ ]:
DRIVE_MOUNTPOINT = "/content/drive"

def mount_drive():
    from google.colab import drive
    drive.mount(DRIVE_MOUNTPOINT, force_remount=True)

_ = !cat {DRIVE_MOUNTPOINT}
if 'No such file or directory' in _[0]:
    mount_drive()

# Workspace Setup

In [ ]:
WORKSPACE = "tf2-object-detection-trainer"

%mkdir -p /content/{WORKSPACE}
%cd /content/{WORKSPACE}

In [ ]:
!pip install python-util tf_slim==1.1.0 lvis==0.5.3 tensorflow-addons==0.12.1 #scipy==1.6.1 pyyaml==5.4.1 gin-config==0.4.0 pandas==1.2.2 matplotlib==3.3.4

In [ ]:
!git clone https://github.com/tensorflow/models tensorflow/models 2> /dev/null
!(cd tensorflow/models;git checkout 73ce096)

In [ ]:
!wget https://github.com/protocolbuffers/protobuf/releases/download/v3.13.0/protoc-3.13.0-linux-x86_64.zip -c
!unzip -u protoc-3.13.0-linux-x86_64.zip -d protoc

In [ ]:
from os import environ as export
_ = !pwd
export['PATH'] += ':' + _[0] + '/protoc/bin'
!(cd tensorflow/models/research;protoc object_detection/protos/*.proto --python_out=.)

In [ ]:
!git clone https://github.com/cocodataset/cocoapi cocoapi 2> /dev/null
!(cd cocoapi;git checkout 8c9bcc3)
!(cd cocoapi/PythonAPI/; printf 'conda activate {ENV_NAME}\nmake\n' | bash -i )
!cp -r ./cocoapi/PythonAPI/pycocotools ./tensorflow/models/research/

In [ ]:
_ = !pwd
TF_OD_PATH = _[0]+'/tensorflow/models/research'

from os import environ as export

export['PYTHONPATH']=TF_OD_PATH+'/:'
export['PYTHONPATH']+=TF_OD_PATH+'/../:'

In [ ]:
!python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

# Directory Structure Setup

In [ ]:
%mkdir -p /content/{WORKSPACE}/targets
%cd /content/{WORKSPACE}/targets

In [ ]:
ANNOTATIONS = 'annotations'
EXPORTED = 'exported-models'
IMAGES = 'images'
TEST_DATA = 'test'
TRAIN_DATA = 'train'
LABEL_MAP = 'label_map.pbtxt'
MODELS = 'models'
PRETRAINED = '.pre-trained-models'
INFERENCE = 'inference'

In [ ]:
!mkdir -p {DRIVE_MOUNTPOINT}/MyDrive/colab/{WORKSPACE}/{TARGET}/{ANNOTATIONS}
!mkdir -p {TARGET}/{ANNOTATIONS}

!echo "" >> {DRIVE_MOUNTPOINT}/MyDrive/colab/{WORKSPACE}/{TARGET}/{ANNOTATIONS}/{LABEL_MAP}
!ln -sf {DRIVE_MOUNTPOINT}/MyDrive/colab/{WORKSPACE}/{TARGET}/{ANNOTATIONS}/{LABEL_MAP} {TARGET}/{ANNOTATIONS}/{LABEL_MAP}

!mkdir -p {TARGET}/{IMAGES}/{TEST_DATA}
!mkdir -p {TARGET}/{IMAGES}/{TRAIN_DATA}

!mkdir -p {DRIVE_MOUNTPOINT}/MyDrive/colab/{WORKSPACE}/{TARGET}/{EXPORTED}
!mkdir -p {DRIVE_MOUNTPOINT}/MyDrive/colab/{WORKSPACE}/{TARGET}/{MODELS}

!ln -sn {DRIVE_MOUNTPOINT}/MyDrive/colab/{WORKSPACE}/{TARGET}/{EXPORTED} {TARGET}/{EXPORTED} 
!ln -sn {DRIVE_MOUNTPOINT}/MyDrive/colab/{WORKSPACE}/{TARGET}/{MODELS} {TARGET}/{MODELS}

!mkdir -p {PRETRAINED}
!mkdir -p /content/train

# Training Data Import

In [ ]:
def import_data():
    !rm -rf {TARGET}/{IMAGES}/dataset

    if DATASET_DOWNLOAD_LINK.endswith('.git'):
        !mkdir {TARGET}/{IMAGES}/dataset/
        !git clone {DATASET_DOWNLOAD_LINK} {TARGET}/{IMAGES}/dataset/

    elif DATASET_DOWNLOAD_LINK.endswith('.tar.gz'):
        !wget {DATASET_DOWNLOAD_LINK} -O {TARGET}/{IMAGES}/dataset.tar.gz -c
        !tar -zxvf {TARGET}/{IMAGES}/dataset.tar.gz -C {TARGET}/{IMAGES} --one-top-level
    
    _ = ! echo {TARGET}/{IMAGES}/dataset/
    export["dataset"]=_[0]
    !cd $dataset && find . -type f -exec sh -c 'new=$(echo "{}" | sed "s/\.\///" | tr "/" "-" | tr " " "_"); mv "{}" "../$new"' \;
    !rm -rf dataset*


import_data()

_ = !grep -Ril "annotation" {TARGET}/{IMAGES} | wc -l

if int(_[0]) < 100:
    raise SystemExit('''
    Import your training data to {}/{} before continuing. 
    You should have at least 100 images for suitable training.
    You may complete the `import_data()` function above to assist you to import your data. 
    '''.format(TARGET, IMAGES))

# Data Preprocessing

In [ ]:
!wget https://raw.githubusercontent.com/sglvladi/TensorFlowObjectDetectionTutorial/725f22217178537030fde9492a7cdb0a7fff4d80/docs/source/scripts/partition_dataset.py -c       

RANDOM_SEED = 10
FRACTION_OF_TEST_DATA = 0.1

from pyutil import filereplace
filereplace("partition_dataset.py", "import random.*", "import random; random.seed({})".format(RANDOM_SEED))

!rm -rf {TARGET}/{IMAGES}/{TRAIN_DATA}/*
!rm -rf {TARGET}/{IMAGES}/{TEST_DATA}/*
!python partition_dataset.py -x -i {TARGET}/{IMAGES} -r {FRACTION_OF_TEST_DATA}

In [ ]:
def get_labels():
    from xml.dom import minidom
    import glob
    objects = set()
    xmls = glob.glob('{}/{}/*.xml'.format(TARGET,IMAGES))

    for xml in xmls:
        xmldoc = minidom.parse(xml)
        itemlist = xmldoc.getElementsByTagName('name')
        for item in itemlist:
            objects.add(item.firstChild.nodeValue)
    return objects

def generate_labemlap():
    global labels
    
    from IPython.core.getipython import get_ipython
    shell = get_ipython()
    content = "%%writefile {TARGET}/{ANNOTATIONS}/{LABEL_MAP}"
    labels = list(get_labels())
    for i in range(len(labels)):
        content+='''
item {{
  id: {}
  name: '{}'
}}'''.format(i+1, labels[i])
    content+="\n\n# generate_labemlap()"
    shell.set_next_input(content, replace=True)
    print("# Template created. Re-run this code-cell to save your labelmap file")
    print(content)

In [ ]:
generate_labemlap()



In [ ]:
# Template created. Re-run this code-cell to save your labelmap file
%%writefile {TARGET}/{ANNOTATIONS}/{LABEL_MAP}
item {
  id: 1
  name: 'Kraft zesty italian dressing'
}
item {
  id: 2
  name: 'mustard'
}
item {
  id: 3
  name: 'Pepsi tin'
}
item {
  id: 4
  name: 'Apple'
}
item {
  id: 5
  name: 'Kraft ranch dressing'
}
item {
  id: 6
  name: 'Cocacola tin'
}
item {
  id: 7
  name: 'Hersheys choco syrup'
}
item {
  id: 8
  name: 'Hersheys strawberry flavoured syrup'
}
item {
  id: 9
  name: 'KitKat Milk Cocoa'
}
item {
  id: 10
  name: 'tomato_ketchup'
}
item {
  id: 11
  name: 'Kelloggs Corn Flakes'
}
item {
  id: 12
  name: 'Hersheys strawberry syrup'
}
item {
  id: 13
  name: 'Loreal Paris Fall Resist'
}
item {
  id: 14
  name: 'Kraft blue cheese dressing'
}
item {
  id: 15
  name: 'Tropicana Guava'
}
item {
  id: 16
  name: 'planters_cheezballs'
}
item {
  id: 17
  name: 'Pampers Pants'
}
item {
  id: 18
  name: 'Colgate Tooth Paste '
}
item {
  id: 19
  name: 'macaroni_cheese'
}
item {
  id: 20
  name: 'Hersheys caramel syrup'
}
item {
  id: 21
  name: 'Cheetos Crunchy'
}
item {
  id: 22
  name: 'Dr pepper tin'
}
item {
  id: 23
  name: 'Thumsup tin'
}
item {
  id: 24
  name: 'Nescafe Classic Coffee'
}
item {
  id: 25
  name: 'Colgate Tooth Paste'
}

# generate_labemlap()



In [ ]:
!wget !wget https://raw.githubusercontent.com/sglvladi/TensorFlowObjectDetectionTutorial/725f22217178537030fde9492a7cdb0a7fff4d80/docs/source/scripts/generate_tfrecord.py -c     

!python generate_tfrecord.py -x {TARGET}/{IMAGES}/{TRAIN_DATA} -l {TARGET}/{ANNOTATIONS}/{LABEL_MAP} -o {TARGET}/{ANNOTATIONS}/train.record
!python generate_tfrecord.py -x {TARGET}/{IMAGES}/{TEST_DATA} -l {TARGET}/{ANNOTATIONS}/{LABEL_MAP} -o {TARGET}/{ANNOTATIONS}/test.record



# Model Import

In [ ]:
MODEL_NAME = MODEL_DOWNLOAD_LINK[MODEL_DOWNLOAD_LINK.rindex('/')+1:MODEL_DOWNLOAD_LINK.index('.tar.gz')]

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/{MODEL_NAME}.tar.gz -P {PRETRAINED} -c
!tar -zxvf {PRETRAINED}/{MODEL_NAME}.tar.gz -C {PRETRAINED}

# Pipeline Configuration

In [ ]:
!rm -rf {TARGET}/{MODELS}/{MODEL_NAME}/train

!mkdir -p {TARGET}/{MODELS}/{MODEL_NAME}/
!cp {PRETRAINED}/{MODEL_NAME}/pipeline.config {TARGET}/{MODELS}/{MODEL_NAME}/
!cp {PRETRAINED}/{MODEL_NAME}/checkpoint/ckpt-0* {TARGET}/{MODELS}/{MODEL_NAME}/

In [ ]:
BATCH_SIZE = 2

In [ ]:
import sys
sys.path.append(TF_OD_PATH)

import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          

with tf.io.gfile.GFile("{}/{}/{}/pipeline.config".format(TARGET,MODELS,MODEL_NAME), "r") as f:                                                                                                                                                                                                                                       
    text_format.Merge(f.read(), pipeline_config)       

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = BATCH_SIZE
pipeline_config.train_config.fine_tune_checkpoint = ""+TARGET+"/"+MODELS+"/"+MODEL_NAME+"/ckpt-0"
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_config.use_bfloat16 = False
pipeline_config.train_input_reader.label_map_path  = ""+TARGET+"/annotations/label_map.pbtxt"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = ""+TARGET+"/"+ANNOTATIONS+"/train.record"   
pipeline_config.eval_input_reader[0].label_map_path  = ""+TARGET+"/annotations/label_map.pbtxt"
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = ""+TARGET+"/"+ANNOTATIONS+"/test.record"                                                                                                                                                                                     
                                                                                                                                                                                                         
with tf.io.gfile.GFile("{}/{}/{}/pipeline.config".format(TARGET,MODELS,MODEL_NAME), "wb") as f:                                                                                                                                                                                                                       
    f.write(text_format.MessageToString(pipeline_config)) 


In [ ]:
!pip install pyutil
from pyutil import filereplace

!cp {TF_OD_PATH}/object_detection/model_main_tf2.py .
!cp {TF_OD_PATH}/object_detection/exporter_main_v2.py .

filereplace("model_main_tf2.py","FLAGS = flags.FLAGS", 
'''flags.DEFINE_boolean('allow_growth', False,
    ('Whether or not to limit gpu memory growth.'))
    
FLAGS = flags.FLAGS

import sys
from signal import signal, SIGPIPE, SIG_DFL 

def receiveSignal(signalNumber, frame):
    print('Received:', signalNumber)
    sys.exit(0)

signal(SIGPIPE, receiveSignal)'''
            
)

filereplace("model_main_tf2.py","if FLAGS.checkpoint_dir:",
'''if FLAGS.allow_growth:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
      except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        pass
        
  if FLAGS.checkpoint_dir:'''
)

# Training Schedule

In [ ]:
# MODEL_NAME = ??? 

if 'MODEL_NAME' not in vars():
    MODEL_NAME = !ls -1 {TARGET}/{MODELS}/
    if len(MODEL_NAME) != 1:
        del MODEL_NAME
        raise SystemExit('''
Could not determine model name. Please specify above.
        ''')
    else:
        MODEL_NAME = MODEL_NAME[0]

MODEL_NAME

In [ ]:
!mkdir -p {TARGET}/{MODELS}/{MODEL_NAME}/train
!mount -o bind /content/train/ {TARGET}/{MODELS}/{MODEL_NAME}/train
!rm -rf /content/train/*

In [ ]:
!rm -rf /content/train/*
!python -u model_main_tf2.py --model_dir={TARGET}/{MODELS}/{MODEL_NAME} \
--pipeline_config_path={TARGET}/{MODELS}/{MODEL_NAME}/pipeline.config \
--allow_growth 2>&1 | sed -e "/loss=nan/q9";echo $? > exitcode

# Checkpoint Validation

In [ ]:
# MODEL_NAME = ??? 

if 'MODEL_NAME' not in vars():
    MODEL_NAME = !ls -1 {TARGET}/{MODELS}/
    if len(MODEL_NAME) != 1:
        del MODEL_NAME
        raise SystemExit('''
Could not determine model name. Please specify above.
        ''')
    else:
        MODEL_NAME = MODEL_NAME[0]

MODEL_NAME

In [ ]:
import sys
sys.path.append(TF_OD_PATH)
sys.path.append(TF_OD_PATH+'/../')

import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import tensorflow as tf
import os

PATH_TO_CFG = "{}/{}/{}/pipeline.config".format(TARGET,MODELS,MODEL_NAME)
PATH_TO_CKPT = "{}/{}/{}".format(TARGET,MODELS,MODEL_NAME)

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore latest checkpoint
latest = next(open("{}/{}/{}/checkpoint".format(TARGET,MODELS,MODEL_NAME)))
latest = latest[latest.index("\"")+1:latest.rindex("\"")]
print(latest)
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, latest)).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
PATH_TO_LABELS = "{}/{}/{}".format(TARGET,ANNOTATIONS,LABEL_MAP)
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
import pathlib
import tensorflow as tf

def download_images():
    base_url = 'file:///content/drive/MyDrive/colab/tf2-object-detection-trainer/{}/{}/'.format(TARGET,INFERENCE)
    filenames = ['image_01.jpg']
    image_paths = []
    for filename in filenames:
        image_path = tf.keras.utils.get_file(fname=filename,
                                            origin=base_url + filename,
                                            untar=False)
        image_path = pathlib.Path(image_path)
        image_paths.append(str(image_path))
    return image_paths

IMAGE_PATHS = download_images()
print(IMAGE_PATHS)

In [ ]:
!rm /root/.keras/datasets/image_01.jpg

In [ ]:
%matplotlib inline
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.30,
            agnostic_mode=False)

    plt.figure(figsize=(10,10))
    
    plt.imshow(image_np_with_detections)
    
    print('Done')
plt.show()

# sphinx_gallery_thumbnail_number = 2

# Model Export

In [ ]:
# MODEL_NAME = ??? 

if 'MODEL_NAME' not in vars():
    MODEL_NAME = !ls -1 {TARGET}/{MODELS}/
    if len(MODEL_NAME) != 1:
        del MODEL_NAME
        raise SystemExit('''
Could not determine model name. Please specify above.
        ''')
    else:
        MODEL_NAME = MODEL_NAME[0]

MODEL_NAME

In [ ]:
!python exporter_main_v2.py --input_type=image_tensor --pipeline_config_path={TARGET}/{MODELS}/{MODEL_NAME}/pipeline.config --trained_checkpoint_dir={TARGET}/{MODELS}/{MODEL_NAME} --output_directory={TARGET}/{EXPORTED}/{MODEL_NAME}
!cp {TARGET}/{ANNOTATIONS}/{LABEL_MAP} {TARGET}/{EXPORTED}/{MODEL_NAME}

# Exported Model Validation

In [ ]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = "{}/{}/{}/saved_model".format(TARGET,EXPORTED,MODEL_NAME)

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
%matplotlib inline
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.5,
          agnostic_mode=False)

    plt.figure(figsize=(10, 10))
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()

# Extras

In [ ]:
%%javascript

var startClickConnect = function startClickConnect(){
    var clickConnect = function clickConnect(){
        console.log("Connnect Clicked - Start");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
        console.log("Connnect Clicked - End"); 
    };

    var intervalId = setInterval(clickConnect, 10000);

    var stopClickConnectHandler = function stopClickConnect() {
        console.log("Connnect Clicked Stopped - Start");
        clearInterval(intervalId);
        console.log("Connnect Clicked Stopped - End");
    };

    return stopClickConnectHandler;
};

var stopClickConnect = startClickConnect();

//stopClickConnect();